In [ ]:
#load packages
import pymc3 as pm
from datetime import datetime, timedelta, time, date
from scipy import stats
import json
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pandas.io import gbq
from google.cloud import bigquery
import plotly.offline as pyo
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import scipy.stats as stats
#import geopandas
from shapely import wkt
import scipy
from scipy import signal
%matplotlib inline
from IPython.core.pylabtools import figsize
from google.colab import drive # to allow colab save file in my drive
drive.mount('/content/drive')
from google.colab import auth
auth.authenticate_user()
print('Authenticated')
cwd=os.getcwd()
cwd
%unload_ext google.colab.data_table
print(f"Running on PyMC3 v{pm.__version__}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Authenticated
The google.colab.data_table extension is not loaded.
Running on PyMC3 v3.7


# load data

In [ ]:
project_id = "logistics-data-staging-flat"
client = bigquery.Client(project = project_id)

query = """
with countries as (
select rdbms_id, entity_id,country_iso 
from `dhh---analytics-apac.pandata.dim_countries`
group by 1,2,3
order by 3
)

select
  entity_id
  , order_code_google
  , cen_order.value.commission_local as central_commission
  , IFNULL(order_commissions.commission_local,
    CASE WHEN fct_orders.country_name in ('Thailand', 'Philippines', 'Hong Kong')
    THEN commission_base_local * commission_percentage_combined
    ELSE fct_orders.commission_local END) as reg_commission,
from `dhh---analytics-apac.pandata.fct_orders` as fct_orders
left join countries co on fct_orders.rdbms_id = co.rdbms_id
left join `dhh---analytics-apac.pandata_report.order_commissions` as order_commissions
         on order_commissions.rdbms_id = fct_orders.rdbms_id
        and order_commissions.order_code = fct_orders.code
        and order_commissions.billing_tool != 'backend'
left join `fulfillment-dwh-production.cl_central_dwh.orders` cen_order on co.entity_id = cen_order.global_entity_id and cen_order.order_id = fct_orders.order_code_google
where is_billable
      AND fct_orders.date_utc between "2021-01-01" and "2021-01-07"
      AND fct_orders.created_date_local < current_date()
group by 1,2,3,4
    """

df = client.query(query).to_dataframe()

In [ ]:
df.head()

,entity_id,order_code_google,central_commission,reg_commission
0,FP_JP,l8jr-ao7i,412,412.0
1,FP_JP,a9qz-85uv,660,660.0
2,FP_JP,g8fl-flqj,2188,2188.0
3,FP_JP,a5vd-z32v,695,695.0
4,FP_JP,b3gk-br9a,234,234.0


## check missing value

In [ ]:
# find out how many orders are without commission 
print("central data", df['central_commission'].isna().sum())
print("regional data", df['reg_commission'].isna().sum())

central data 220
regional data 0


In [ ]:
df = df.dropna()
df['central_commission'] = df['central_commission'].astype(float)

In [ ]:
df.describe()

,central_commission,reg_commission
count,1.352994e+07,1.352994e+07
mean,1.334658e+02,1.344991e+02
std,1.144202e+03,1.144126e+03
min,-2.322000e+03,-2.322000e+03
25%,9.730000e+00,1.144000e+01
50%,2.900000e+01,3.000000e+01
75%,5.607000e+01,5.753571e+01
max,7.632000e+05,7.632000e+05


## check unequal ones

In [ ]:
df['equal'] = np.where(df['central_commission'] == df['reg_commission'], True, False)

In [ ]:
equal = df.pivot_table("order_code_google",index="entity_id",columns="equal",aggfunc="count").reset_index()
equal = equal.fillna(0)
equal["total"] = equal[False] + equal[True]
equal["false_percentage"] = equal[False]/equal["total"]

In [ ]:
fig = px.bar(equal, x='entity_id', y='false_percentage')
fig.show()

In [ ]:
df['equal'].value_counts()

True     9840425
False    3689510
Name: equal, dtype: int64

## Check descriptive data

In [ ]:
df_melt = pd.melt(df, id_vars = ["entity_id","order_code_google"], value_vars = ["central_commission", "reg_commission"],var_name = "source", value_name = "commission")

In [ ]:
df_melt.head()

,entity_id,order_code_google,source,commission
0,FP_JP,l8jr-ao7i,central_commission,412.0
1,FP_JP,a9qz-85uv,central_commission,660.0
2,FP_JP,g8fl-flqj,central_commission,2188.0
3,FP_JP,a5vd-z32v,central_commission,695.0
4,FP_JP,b3gk-br9a,central_commission,234.0


In [ ]:
entities = df_melt.entity_id.unique().tolist()
sources = df_melt.source.unique().tolist()

In [ ]:
# create empty array list
rows_list = []

for entity in entities:
  for source in sources:
    df_agg = df_melt[df_melt["entity_id"] == entity]
    df_agg = df_agg[df_agg["source"] == source]
    sum = df_agg["commission"].sum()
    mean = df_agg["commission"].mean()
    std = df_agg["commission"].std()
    min = df_agg["commission"].min()
    quantile25 = df_agg["commission"].quantile(.25) 
    median = df_agg["commission"].median()
    quantile75 = df_agg["commission"].quantile(.75) 
    max = df_agg["commission"].max() 
    rows_list.append([entity, source,sum,mean,
                      std,
                      min, quantile25,median,quantile75,max])

result = pd.DataFrame(rows_list, columns=['entity','source','sum','mean','std',
                                          'min','quantile25','median','quantile75','max'])
result

,entity,source,sum,mean,std,min,quantile25,median,quantile75,max
0,FP_JP,central_commission,1.491862e+07,454.351241,294.228630,0.000000,273.00,383.0000,546.0000,6583.000000
1,FP_JP,reg_commission,1.491862e+07,454.351241,294.228630,0.000000,273.00,383.0000,546.0000,6583.000000
2,FP_BD,central_commission,1.672689e+07,53.392601,75.251184,0.000000,12.10,36.4000,64.4000,4510.000000
3,FP_BD,reg_commission,1.672121e+07,53.374478,75.234645,0.000000,12.10,36.4000,64.4000,4510.000000
4,FP_HK,central_commission,2.235594e+07,39.906864,49.974692,0.000000,19.20,30.2400,47.5200,3358.320000
5,FP_HK,reg_commission,2.235594e+07,39.906864,49.974692,0.000000,19.20,30.2400,47.5200,3358.320000
6,FP_KH,central_commission,1.507708e+05,1.106818,0.944394,-0.080000,0.53,0.8300,1.3500,31.540000
7,FP_KH,reg_commission,1.507708e+05,1.106818,0.944394,-0.080000,0.53,0.8300,1.3500,31.540000
8,FP_LA,central_commission,9.895177e+08,9854.674199,8672.365659,0.000000,4500.00,7500.0000,12000.0000,763200.000000
9,FP_LA,reg_commission,9.895177e+08,9854.674199,8672.365659,0.000000,4500.00,7500.0000,12000.0000,763200.000000
